In [ ]:
import numpy as np
import math
import os
import matplotlib.pyplot as plt
from google.colab import drive
from PIL import Image

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive')
img = Image.open('pepper.png')
img = np.array(img)

In [ ]:
def cluster(point, data_point, radius):
  x = point[0]
  y = point[1]
  a = data_point[0]
  b = data_point[1]

  distance = math.sqrt((x-a)**2 + (y-b)**2)
  if(distance > radius):
    return 0
  else:
    return 1
  
def mean_shifting(data, radius):
  data /= np.amax(data,axis=0)
  cluster_array = np.full((row*col), -1)
  id = 0

  for i in range(len(data)):
    if(cluster_array[i] != -1):
      continue
    else:
      centroid = data[i]

      while(True):
        neighborhood = data[np.sum((data - centroid)**2, axis=1) < radius**2]
        neighborhood = neighborhood[cluster_array[np.where(np.sum((data - centroid)**2, axis=1) < radius**2)] == -1]
        new_centroid = np.mean(neighborhood,axis=0)

        if(np.sum((new_centroid - centroid)**2) < 0.1):
          cluster_array[np.sum((data - centroid)**2, axis=1) < radius**2] = id
          id += 1
          break

        centroid = new_centroid
  return data, cluster_array,id

In [ ]:
row = np.shape(img)[0]
col = np.shape(img)[1]

data = np.zeros((row* col, 5))  # create an empty array of the desired shape and data type
for i in range(row):
  for j in range(col):
    # append the RGB values and the coordinates to the new array
    data[i*col + j,0:3] = img[i,j,0:3]
    data[i*col + j,3:5] = [i,j]

data, clusters,num = mean_shifting(data,0.1)

In [ ]:
for i in range(0,num-1):
  data[clusters==i] = data[np.where(clusters==i)[0][0]]

In [ ]:
data = data.reshape(row,col,5)
r = data[...,0]
g = data[...,1]
b = data[...,2]
plt.imshow(np.stack((r,g,b), axis=-1)) 